# Pipelines Benchmarking :

- The methodology is so straightforward, first we should counstarct our grounding truth small dataset of extracted informations from Invoices, these can be done using GPT4 because is so good on that.
- Then we can compare each approach outputs with the grounding truth, and calculate **Accuaracy**.
- Three Benchmarks are Considered: `Accuracy`, `Inference time (s)` and `Cost`.
- The Grounding Truth dataset will be in a form of a csv file we will focus on Entities like :

    - TT,TTC,TVA,Date de facture,Numero facture, Supplier Name.

- All the Bechmarking process is done with a L4 machine.(But you Can try it with T4 😄)



## Setup Ollama

In [1]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [62.5 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,234 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,611 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,228 kB]
Hit:13 https://ppa.launchp

In [2]:
!ollama list

NAME    ID    SIZE    MODIFIED 


## Grounding Truth:

In [3]:
!apt-get install git
!git init
!git remote add origin https://github.com/MasrourTawfik/Textra_Insights.git
!git config core.sparseCheckout true
!echo "Files/" >> .git/info/sparse-checkout
!git pull origin main

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 57 not upgraded.
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 854, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 854 (delta 151), reused 175 (delta 81), pack-reused 599 (from 1)
Receiving objects: 100% (85

In [4]:
import pandas as pd
# Path to your CSV file from drive
csv_file_path = "Files/grounding_truth.csv"
df = pd.read_csv(csv_file_path)
df.head()

,Invoice_Name,Fournisseur,Date_Facture,Num_Facture,TTC,TT,TVA
0,2.jpg,KIPRINT,13/01/2023,FV230007,586.80,489.00,97.80
1,3.jpg,FUNPLACE,14/04/2023,FUNPLACE - 2023/000403,3864.00,3220.00,644.00
2,49.jpg,LYDEC,30/03/2023,1507042573 AM,224.28,209.59,14.69
3,57.jpg,ARCANES TECHNOLOGIES,04-01-2023,223285-23-DJ,226.80,189.00,37.80
4,58.jpg,NETTOPAP,10/03/2023,230058,3026.45,2522.04,504.41


# Pipeline 1 :

## OCR

- A Blog by <a href="https://blog.roboflow.com/best-ocr-models-text-recognition/#ocr-testing-methodology">Roboflow</a> countain a good bechamrking of 4 OCR models (docTR, EasyOCR, Tesseract, Surya) Show that EasyOCR has the best accuarcy/speed ratio.
- Another comparaison between EasyOCR and <a href = "https://paddlepaddle.github.io/PaddleOCR/latest/en/index.html">PaddleOCR</a> by **Christian Weiler** <a href="https://www.bludelta.de/en/ocr-and-deepocr-in-comparison/">here</a> on 400 real invoices, show that EasyOCR with a GPU has the best accuracy/speed ratio.
- <a href="https://www.geeksforgeeks.org/introduction-to-levenshtein-distance/">Levenshtein distance</a>

### Install Paddle and Easy OCR

Paddle Installation :
```bash
!pip install paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install "paddleocr>=2.0.1" # Recommend to use version 2.0.1+
```

EasyOCR Installation :
```bash
!pip install easyocr -q
```

In [5]:
!pip install paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install "paddleocr>=2.0.1" # Recommend to use version 2.0.1+

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 122.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 53.1 kB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.7/544.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 99.2 MB/s eta 0:00:00
   

In [6]:
!pip install easyocr -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.6/286.6 kB 24.3 MB/s eta 0:00:00


### Run OCR on All Invoices :

- PaddleOCR
- EasyOCR (Switch to T4 so We don't wait alot)

#### OCR_Functions

In [7]:
import time
import os
from tqdm import tqdm
import pandas as pd

################################################################################
################################### PaddleOCR ##################################
from paddleocr import PaddleOCR
paddle_reader = PaddleOCR(use_angle_cls=True, lang='fr')

def Run_Paddle(Image_path):
    Start = time.time()
    result = paddle_reader.ocr(Image_path, det=True, rec=True)
    End = time.time()
    # Inference Time
    Time = End-Start
    # Process the result
    extracted_text = ''
    #print("Lenght Result: ",len(result))
    for line in result:
        if line is not None:
            for word in line:
                #print("Word: ",word)
                # Access the text part of the tuple
                extracted_text += word[1][0] + ' '  # Access the first element of the recognized text (the text itself)
            extracted_text += '\n'
    return extracted_text,Time


def Run_Paddle_ALL(Folder_path):
    df = pd.DataFrame(columns=['InvoiceName', 'Text', 'Time(s)'])
    Files = os.listdir(Folder_path)
    Files = sorted(Files, key=lambda x: int(x.split('.')[0]))
    Texts = []
    Times = []
    for file in tqdm(Files,desc="Paddle Processing", unit="number"):
        print(f"{file} is processing")
        Invoice_path = os.path.join(Folder_path, file)
        Text_paddle,Time_paddle = Run_Paddle(Invoice_path)
        Texts.append(Text_paddle)
        Times.append(Time_paddle)
    df['InvoiceName'] = Files
    df['Text'] = Texts
    df['Time(s)'] = Times

    # Saving the DataFrame to a CSV file
    df.to_csv('CSVs/PaddleOCR_Results.csv', index=False)

################################################################################
################################### EasyOCR ####################################

import easyocr
easyocr_reader = easyocr.Reader(['fr'],gpu = True)

def Run_EasyOCR(Image_path):
    Start = time.time()
    result = easyocr_reader.readtext(Image_path)
    End = time.time()

    # Inference Time
    Time = End-Start

    extracted_text = ''
    for line in result:
        extracted_text += line[1] + ' '
    return extracted_text,Time


def Run_EasyOCR_ALL(Folder_path):
    df = pd.DataFrame(columns=['InvoiceName', 'Text', 'Time(s)'])
    Files = os.listdir(Folder_path)
    Files = sorted(Files, key=lambda x: int(x.split('.')[0]))
    Texts = []
    Times = []
    for file in tqdm(Files,desc="EasyOCR Processing", unit="number"):
        print(f"{file} is processing")
        Invoice_path = os.path.join(Folder_path, file)
        Text_easyocr,Time_easyocr = Run_EasyOCR(Invoice_path)
        Texts.append(Text_easyocr)
        Times.append(Time_easyocr)
    df['InvoiceName'] = Files
    df['Text'] = Texts
    df['Time(s)'] = Times
    # Saving the DataFrame to a CSV file
    df.to_csv('CSVs/EasyOCR_Results.csv', index=False)




download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:00<00:00, 5141.27it/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/latin_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer/latin_PP-OCRv3_rec_infer.tar


100%|██████████| 9930/9930 [00:05<00:00, 1933.82it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:00<00:00, 3000.12it/s]

[2025/01/18 17:12:11] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25,

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

#### Test

In [8]:
import os

os.makedirs("CSVs", exist_ok=True)
Folder_path = "Files/Test_Invoices"

In [9]:
Run_Paddle_ALL(Folder_path)

Paddle Processing:   0%|          | 0/30 [00:00<?, ?number/s]

2.jpg is processing
[2025/01/18 17:12:33] ppocr DEBUG: dt_boxes num : 54, elapsed : 0.6882009506225586
[2025/01/18 17:12:34] ppocr DEBUG: cls num  : 54, elapsed : 0.26166200637817383
[2025/01/18 17:12:38] ppocr DEBUG: rec_res num  : 54, elapsed : 4.228612661361694


Paddle Processing:   3%|▎         | 1/30 [00:05<02:34,  5.32s/number]

3.jpg is processing
[2025/01/18 17:12:38] ppocr DEBUG: dt_boxes num : 36, elapsed : 0.3020820617675781
[2025/01/18 17:12:38] ppocr DEBUG: cls num  : 36, elapsed : 0.14070415496826172
[2025/01/18 17:12:41] ppocr DEBUG: rec_res num  : 36, elapsed : 2.67647385597229


Paddle Processing:   7%|▋         | 2/30 [00:08<01:53,  4.07s/number]

...

Paddle Processing:  93%|█████████▎| 28/30 [01:22<00:04,  2.41s/number]

417.jpg is processing
[2025/01/18 17:13:56] ppocr DEBUG: dt_boxes num : 95, elapsed : 0.405609130859375
[2025/01/18 17:13:56] ppocr DEBUG: cls num  : 95, elapsed : 0.25420379638671875
[2025/01/18 17:13:58] ppocr DEBUG: rec_res num  : 95, elapsed : 2.140789747238159


Paddle Processing:  97%|█████████▋| 29/30 [01:25<00:02,  2.54s/number]

419.jpg is processing
[2025/01/18 17:13:59] ppocr DEBUG: dt_boxes num : 41, elapsed : 0.28917956352233887
[2025/01/18 17:13:59] ppocr DEBUG: cls num  : 41, elapsed : 0.10083651542663574
[2025/01/18 17:14:00] ppocr DEBUG: rec_res num  : 41, elapsed : 1.0996325016021729


Paddle Processing: 100%|██████████| 30/30 [01:27<00:00,  2.91s/number]


In [10]:
Run_EasyOCR_ALL(Folder_path)

EasyOCR Processing:   0%|          | 0/30 [00:00<?, ?number/s]

2.jpg is processing


EasyOCR Processing:   3%|▎         | 1/30 [00:03<01:54,  3.96s/number]

3.jpg is processing


EasyOCR Processing:   7%|▋         | 2/30 [00:06<01:22,  2.95s/number]

...

EasyOCR Processing:  93%|█████████▎| 28/30 [01:22<00:05,  2.72s/number]

417.jpg is processing


EasyOCR Processing:  97%|█████████▋| 29/30 [01:24<00:02,  2.67s/number]

419.jpg is processing


EasyOCR Processing: 100%|██████████| 30/30 [01:27<00:00,  2.91s/number]


In [13]:
import pandas as pd
df_paddle = pd.read_csv('CSVs/PaddleOCR_Results.csv')
df_paddle.head()

,InvoiceName,Text,Time(s)
0,2.jpg,FACTURE N°: FV230007 KiPRiNT Date Délai Livra...,5.317665
1,3.jpg,Facture N°:FUNPLACE -2023/000403 FUNART Date:1...,3.187335
2,49.jpg,Référence : 0026757005: Votre Direction Préfe...,6.987693
3,57.jpg,Page : 1/1 ARCANES TECHNOLOGIES FACTURE: 22328...,4.991912
4,58.jpg,NETTOOPAP PRODUITS DE NETTOYAGE ET D'HYGIENE S...,4.107593


In [14]:
df_easyocr = pd.read_csv('CSVs/EasyOCR_Results.csv')
df_easyocr.head()

,InvoiceName,Text,Time(s)
0,2.jpg,FACTURE N' FV230007 KiPRINT Date Délai Livrais...,3.957948
1,3.jpg,"he Facture N"": FUNPLACE 2023/000403 FUN ART Da...",2.232184
2,49.jpg,Référence 0026 757 005 Votre Direction Préfect...,4.691591
3,57.jpg,ensombis Oisonsc Waroc(ehnoogguedrtaog Page 1 ...,1.944791
4,58.jpg,NETTO PAP PRODUITS DE NETTOYAGE ET D'HYGIÈNE S...,3.785860


## LLM

- From a quick search, you can find that LLM via API Providers like OpenAI, Anthropic... are the best in Information extraction.
But since we decide to keep everything localy, we need to test some of the local LLMs with Oallama and Huggingface.

- For All the tested models i use the q-4 quantized version of the model.

#### Install Dependencies

In [ ]:
!pip install langchain-openai==0.2.9

In [16]:
!ollama pull llama3.2:3b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...  

In [17]:
!ollama list

NAME           ID              SIZE      MODIFIED      
llama3.2:3b    a80c4f17acd5    2.0 GB    8 seconds ago    


#### LLM_Functions

In [21]:
################################################################################
################################ Structered Output ################################
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class Extraction_Facture(BaseModel):
    """Les Informations principales à extraire a partir d'une facture française"""
    Nom_facture: str = Field("", description="Le nom et le format(.jpg, .png, .jpeg) de l'image de la facture")
    Fournisseur: str = Field("", description="Le nom du fournisseur de la facture")
    date_facture: str = Field("", description="La date de la facture")
    num_facture: str = Field("", description="L'identifiant de la facture, peut être numérique ou alphanumérique")
    ttc: float = Field(0.0, description="Toutes Taxes Comprises. Il s'agit du prix final total payé par le client. Il inclut le prix Total HT et la Total TVA.")
    tt: float = Field(0.0, description="Le total hors taxes de la facture")
    tva: float = Field(0.0, description="Le Total taxes")



system = """
Vous êtes un assistant puissant dans l'extraction d'informations a partir des factures francaises. \n
- Un OCR est appliqué sur l'image de la facture, lire le text et extraire les informations suivantes :  \n

- Fournisseur, Date_Facture, Num_Facture, TTC, TT, TVA \n
- Les 3 prix TT,TTC,TVA doivent être numérique sans indication de devise. \n
- le nom de Fournisseur doit être en majuscule. \n

## Important :

- Vérifiez toujours que TT + TVA = TTC, pour assurer que les prix extraits sont corrects.

"""

## Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Le text donné par OCR ici :\n {OCR_Output} \n le Nom de facture est : \n {Nom_facture} \n"),
    ]
)

################################################################################
################################ LLM ###########################################
from langchain_openai import ChatOpenAI
import pandas as pd
import time

LLM = ChatOpenAI(
    model="llama3.2:3b",
    #temperature=0,
    api_key='ollama',
    base_url="http://localhost:11434/v1",
)
LLM_Structered = LLM.with_structured_output(Extraction_Facture)
LLM_Extractor = prompt | LLM_Structered

def Run_LLM(OCR_Output, Nom_facture):
    Input = {"OCR_Output":OCR_Output, "Nom_facture":Nom_facture}
    response = LLM_Extractor.invoke(Input)
    return response

# The Chain can raise an exception if the LLM fails to respect the structured output, so we can fix a retry number
RETRIES = 3

def Run_LLM_ALL(OCR_dataframe,pipelineName):
    LLM_Results = pd.DataFrame(columns=['Temps(s)','Nom_facture', 'Fournisseur', 'Date_Facture', 'Num_Facture', 'TTC', 'TT', 'TVA'])
    for index, row in OCR_dataframe.iterrows():
        print(f"Processing {index+1}/{len(OCR_dataframe)}...")
        for i in range(RETRIES):
            try:
                strat_time = time.time()
                response = Run_LLM(row['Text'], row['InvoiceName'])
                end_time = time.time()
                response_time = end_time - strat_time
                LLM_Results.loc[index] = [response_time,row['InvoiceName'], response.Fournisseur, response.date_facture, response.num_facture, response.ttc, response.tt, response.tva]
                #print(LLM_Results)
                break
            except Exception as e:
                #print(f"Error: {e}")
                LLM_Results.loc[index] = [0.0,row['InvoiceName'], '', '', '', 0.0, 0.0, 0.0]
                print(f"Retrying {i+1}/{RETRIES}...\n")

    # Saving the DataFrame to a CSV file
    LLM_Results.to_csv(f'CSVs/{pipelineName}_Results.csv', index=False)

#### Load OCR Results

In [22]:
df_paddle = pd.read_csv('CSVs/PaddleOCR_Results.csv')
df_paddle.head()

,InvoiceName,Text,Time(s)
0,2.jpg,FACTURE N°: FV230007 KiPRiNT Date Délai Livra...,5.317665
1,3.jpg,Facture N°:FUNPLACE -2023/000403 FUNART Date:1...,3.187335
2,49.jpg,Référence : 0026757005: Votre Direction Préfe...,6.987693
3,57.jpg,Page : 1/1 ARCANES TECHNOLOGIES FACTURE: 22328...,4.991912
4,58.jpg,NETTOOPAP PRODUITS DE NETTOYAGE ET D'HYGIENE S...,4.107593


In [23]:
df_easyocr = pd.read_csv('CSVs/EasyOCR_Results.csv')
df_easyocr.head()

,InvoiceName,Text,Time(s)
0,2.jpg,FACTURE N' FV230007 KiPRINT Date Délai Livrais...,3.957948
1,3.jpg,"he Facture N"": FUNPLACE 2023/000403 FUN ART Da...",2.232184
2,49.jpg,Référence 0026 757 005 Votre Direction Préfect...,4.691591
3,57.jpg,ensombis Oisonsc Waroc(ehnoogguedrtaog Page 1 ...,1.944791
4,58.jpg,NETTO PAP PRODUITS DE NETTOYAGE ET D'HYGIÈNE S...,3.785860


In [24]:
index = 2
Text_paddle = df_paddle['Text'].values[index]
Text_easyocr = df_easyocr['Text'].values[index]
InvoiceName = df_paddle['InvoiceName'].values[index]

print("Text paddle :", Text_paddle)
print("Text easyocr :", Text_easyocr)
print("InvoiceName :", InvoiceName)

response = Run_LLM(Text_paddle, InvoiceName)
response

Text paddle : Référence :  0026757005: Votre Direction Préfectorale CASA-ANFA Facture Eau N° 1507042573AM p xbillg cl 8gL 48, Rue Mohamed Diouri Date 30/03/2023 CASABLANCA contact@lydec.co.ma ICE: 001685924000018 www.lydec.ma Lydec lw Centre de relation clientéle 0522312020 P 202304 SMART CALL DISTRIBUTION 3 RUE ABDELKADER ELMAZINI ETG 1 APPT GH CASABLANCA Avis de facturation Eau et slal 3 y g&L yle&) assainissement 18/02/2023 au 22/03/2023 C9 1m9KF 1M3 Période de consommation Eau du Evolution de votre consommation pyal4 lll Consommation Montants en Dhs Eau et assainissement 14 m3 209,59 Jx aifl g s atl Taxes 14,69 Avr Mai Jui Jul AoaSepOct NovDécJan Fév Mar Avr Sous total TTC 224,28 22222222222222222223232323 Votre contrat Net ä payer 224,28 1171819 Mode de réglement Prélévement bancaire Prochain passage de votre encaisseur Votre option tarifaire Montant des impayés TTC ** 0,00 1P? o 1p6r19 q-Activité commerciale et pro  Sauf erreur ou omission 8ui gf Uas Isc Date limite de paiement 1

Extraction_Facture(Nom_facture='49.jpg', Fournisseur='LYDEC SA', date_facture='30/03/2023', num_facture='1507042573AM', ttc=224.28, tt=0.0, tva=14.69)

#### Run LLM on All Invoices

In [25]:
pipelineName = "llama3.2_3b_Paddle"
Run_LLM_ALL(df_paddle,pipelineName)
#pipelineName = "Qwen2.5_7b_Easy"
#Run_LLM_ALL(df_easyocr,pipelineName)

Processing 1/30...
Processing 2/30...
Processing 3/30...
Retrying 1/3...

Processing 4/30...
Retrying 1/3...

Processing 5/30...
Retrying 1/3...

Processing 6/30...
Retrying 1/3...

Retrying 2/3...

Processing 7/30...
Processing 8/30...
Processing 9/30...
Processing 10/30...
Processing 11/30...
Retrying 1/3...

Processing 12/30...
Processing 13/30...
Processing 14/30...
Processing 15/30...
Processing 16/30...
Processing 17/30...
Processing 18/30...
Retrying 1/3...

Processing 19/30...
Processing 20/30...
Retrying 1/3...

Retrying 2/3...

Retrying 3/3...

Processing 21/30...
Processing 22/30...
Retrying 1/3...

Processing 23/30...
Processing 24/30...
Retrying 1/3...

Processing 25/30...
Processing 26/30...
Retrying 1/3...

Processing 27/30...
Processing 28/30...
Processing 29/30...
Processing 30/30...


#### Evaluation Function

In [28]:
!pip install python-Levenshtein

In [29]:
import Levenshtein
import pandas as pd

def Calculate_Similarity(string1, string2):
    if pd.isna(string1):
        return 0
    distance = Levenshtein.distance(string1, string2)
    score = 1-(distance/max(len(string1), len(string2)))
    return score

def Evaluate(LLM_Results, Ground_Truth):
    Total_Similarity = 0
    Total_Margin = 0
    Total_Time = 0
    for index, Ground_row in Ground_Truth.iterrows():
        print("######## Invoice Number : ", index)
        Line_Similarity = 0
        Line_margin = 0

        # Strings Similarities
        Line_Similarity = Calculate_Similarity(LLM_Results.loc[index,'Fournisseur'], Ground_row['Fournisseur']) + Calculate_Similarity(LLM_Results.loc[index,'Date_Facture'], Ground_row['Date_Facture']) + Calculate_Similarity(LLM_Results.loc[index,'Num_Facture'], Ground_row['Num_Facture'])
        Total_Similarity += Line_Similarity/3
        print("Total Similarity : ", Total_Similarity)

        # Prices Margins
        Line_margin = abs(LLM_Results.loc[index,'TTC'] - Ground_row['TTC']) + abs(LLM_Results.loc[index,'TT'] - Ground_row['TT']) + abs(LLM_Results.loc[index,'TVA'] - Ground_row['TVA'])
        Total_Margin += Line_margin
        print("Total Margin : ", Total_Margin)

        Total_Time += LLM_Results.loc[index,'Temps(s)']

    Accuarcy = Total_Similarity/len(Ground_Truth)
    Average_Time = Total_Time/len(Ground_Truth)
    Average_Margin = Total_Margin/len(Ground_Truth)
    return Accuarcy, Average_Time, Average_Margin


#### Evaluation of Performance

In [ ]:
LLM_Results = pd.read_csv(f'CSVs/{pipelineName}_Results.csv')
Ground_Truth = pd.read_csv('Files/grounding_truth.csv')
Accuarcy, Average_Time, Average_Margin = Evaluate(LLM_Results, Ground_Truth)

In [34]:
print("Accuarcy :(%) ", Accuarcy*100)
print("Average_Time(s) : ", Average_Time)
print("Average_Margin (DH) : ", Average_Margin)

Accuarcy :(%)  69.99765711883879
Average_Time(s) :  1.5326431751251222
Average_Margin (DH) :  7621.832166666664


- Run the next cell Only for the fist time to define the dataframe.

In [35]:
#Results= pd.DataFrame(columns=['PipelineName','Accuarcy (%)', 'Average_Time (s)', 'Average_Margin (DH)'])
#Results.to_csv('CSVs/Results.csv', index=False)

In [36]:
Results = pd.read_csv('CSVs/Results.csv')
Results

,PipelineName,Accuarcy (%),Average_Time (s),Average_Margin (DH)


In [37]:
Results.loc[len(Results)] = [pipelineName,Accuarcy*100, Average_Time, Average_Margin]
Results.to_csv('CSVs/Results.csv', index=False)

In [38]:
Results

,PipelineName,Accuarcy (%),Average_Time (s),Average_Margin (DH)
0,llama3.2_3b_Paddle,69.997657,1.532643,7621.832167


# Pipeline 2 :

- In this pipeline we will use Vision language models (Multimodal LLM) to diracetly extract key entities from the invoice image.
- With Local Models the reulsts are so poor, the VLM can't even respect the system prompt, and perform alot of mistakes bigger models may perform better
- We worked here with Image encoding instead of Url because we will use Local VLMs.

## Convert Images to a base64 string

In [9]:
import base64
import os
import pandas as pd
from tqdm import tqdm

In [40]:
def Image_to_base64(image_path):
    try:
        with open(image_path, "rb") as image_file:
            image_binary = image_file.read()
            base64_encoded = base64.b64encode(image_binary)
            return base64_encoded.decode("utf-8")
    except FileNotFoundError:
        return "Error: File not found."
    except Exception as e:
        return f"Error: {e}"

def Encode_All(Folder_path):
    Files = os.listdir(Folder_path)
    Files = sorted(Files, key=lambda x: int(x.split('.')[0]))
    Encoding_df = pd.DataFrame(columns=['InvoiceName', 'Image_Base64'])
    for file in tqdm(Files,desc="Encoding", unit="number"):
        print(f"{file} is processing")
        Invoice_path = os.path.join(Folder_path, file)
        base64_encoded = Image_to_base64(Invoice_path)
        Encoding_df.loc[len(Encoding_df)] = [file, base64_encoded]
    # Saving the DataFrame to a CSV file
    Encoding_df.to_csv('CSVs/Encoding.csv', index=False)

- Example :

In [41]:
Image_path = "Files/Test_Invoices/2.jpg"
encoding = Image_to_base64(Image_path)
encoding

'...'

In [42]:
Encode_All("Files/Test_Invoices")

Encoding: 100%|██████████| 30/30 [00:00<00:00, 394.92number/s]


2.jpg is processing
3.jpg is processing
49.jpg is processing
57.jpg is processing
58.jpg is processing
61.jpg is processing
66.jpg is processing
67.jpg is processing
72.jpg is processing
76.jpg is processing
109.jpg is processing
114.jpg is processing
121.jpg is processing
129.jpg is processing
131.jpg is processing
151.jpg is processing
164.jpg is processing
307.jpg is processing
311.jpg is processing
314.jpg is processing
315.jpg is processing
316.jpg is processing
317.jpg is processing
319.jpg is processing
327.jpg is processing
334.jpg is processing
340.jpg is processing
383.jpg is processing
417.jpg is processing
419.jpg is processing


In [10]:
Encoding_df = pd.read_csv('CSVs/Encoding.csv')
Encoding_df.head()

,InvoiceName,Image_Base64
0,2.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,3.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,49.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,57.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,58.jpg,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


## Benchamrking

#### Install Dependencies

In [44]:
exit() # To clear the GPU RAM (VRAM)

In [3]:
!ollama list

NAME           ID              SIZE      MODIFIED      
llama3.2:3b    a80c4f17acd5    2.0 GB    5 minutes ago    


In [ ]:
!ollama pull llava-llama3:8b

In [5]:
!ollama list

NAME               ID              SIZE      MODIFIED      
llava-llama3:8b    44c161b1f465    5.5 GB    8 seconds ago    
llama3.2:3b        a80c4f17acd5    2.0 GB    7 minutes ago    


#### VLM_Functions

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


system = """
Vous êtes un assistant puissant dans l'extraction d'informations a partir des factures francaises. \n

## Les Informations à extraire sont : \n

- Observer bien la facture et extraire les informations suivantes :  \n

   - Nom_facture : Un String, Le nom et le format(.jpg, .png, .jpeg) de l'image de la facture. \n
   - Fournisseur : Un String, Le nom du fournisseur de la facture. \n
   - date_facture : Un String, La date de la facture. \n
   - num_facture : Un String, L'identifiant de la facture, peut être numérique ou alphanumérique. \n
   - ttc : Un Float, Toutes Taxes Comprises. Il s'agit du prix final total payé par le client. Il inclut le prix Total HT et la Total TVA. \n
   - tt : Un Float, Le total hors taxes de la facture. \n
   - tva : Un Float, Le Total taxes. \n

## Structure de la réponse : \n

- Votre reponse doit être un dictionnaire python.

## Consignes : \n

- Les 3 prix TT,TTC,TVA doivent être numérique sans indication de devise. \n
- le nom de Fournisseur doit être en majuscule. \n
"""
## Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "user",
            [
                 {
                    "type": "text",
                    "text": system,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": "data:image/jpeg;base64,{image_data}"},
                }
            ],
        ),
    ]
)

VLM = ChatOpenAI(
    model="llava-llama3:8b",
    #temperature=0,
    api_key='ollama',
    base_url="http://localhost:11434/v1",
)

VLM_Extractor = prompt | VLM

def Run_VLM(InvoiceName,image_data):
    response = VLM_Extractor.invoke({"image_data": image_data,"InvoiceName": InvoiceName})
    return response

#### Test

In [12]:
encoding = Encoding_df.loc[0].Image_Base64
Name = Encoding_df.loc[0].InvoiceName
response = Run_VLM(Name,encoding)
response

AIMessage(content="The image presents a black and white invoice issued by Kiprint, a company specializing in photo printing services. The invoice is signed by the director of the company, indicating its authenticity.\n\nThe main elements of the invoice include the name and contact information of the customer, as well as the details of the order placed with the company. The order includes one copy of a photo album, priced at 760€, and one CD-ROM, priced at 150€. The total cost for the items ordered is 910€.\n\nThe invoice also specifies that the payment method was by credit card and the date of the transaction is May 18th, 2012. The company's contact information is provided on the bottom right corner of the invoice. \n\nThis detailed breakdown provides a comprehensive overview of the customer's purchase history with Kiprint, including the items ordered, their prices, the payment method, and the date of the transaction.", additional_kwargs={'refusal': None}, response_metadata={'token_usa

In [13]:
print(response.content)

The image presents a black and white invoice issued by Kiprint, a company specializing in photo printing services. The invoice is signed by the director of the company, indicating its authenticity.

The main elements of the invoice include the name and contact information of the customer, as well as the details of the order placed with the company. The order includes one copy of a photo album, priced at 760€, and one CD-ROM, priced at 150€. The total cost for the items ordered is 910€.

The invoice also specifies that the payment method was by credit card and the date of the transaction is May 18th, 2012. The company's contact information is provided on the bottom right corner of the invoice. 

This detailed breakdown provides a comprehensive overview of the customer's purchase history with Kiprint, including the items ordered, their prices, the payment method, and the date of the transaction.


# Pipeline 3 :

## Parser

#### Parser_Functions

In [ ]:
!pip install docling

In [2]:
from paddleocr import PaddleOCR
paddle_reader = PaddleOCR(use_angle_cls=True, lang='fr')

def Run_Paddle(Image_path):
    Start = time.time()
    result = paddle_reader.ocr(Image_path, det=True, rec=True)
    End = time.time()
    # Inference Time
    Time = End-Start
    # Process the result
    extracted_text = ''
    #print("Lenght Result: ",len(result))
    for line in result:
        if line is not None:
            for word in line:
                #print("Word: ",word)
                # Access the text part of the tuple
                extracted_text += word[1][0] + ' '  # Access the first element of the recognized text (the text itself)
            extracted_text += '\n'
    return extracted_text,Time
#############################################################################
############################## Docling v2 ###################################

from docling_core.types.doc import PictureItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions,EasyOcrOptions
from docling.document_converter import DocumentConverter,PdfFormatOption
import time

pipeline_options = PdfPipelineOptions()

pipeline_options.images_scale = 1
#pipeline_options.generate_page_images = True
#pipeline_options.generate_table_images = True
pipeline_options.do_ocr = True
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = True
ocr_options = EasyOcrOptions(force_full_page_ocr=True)
pipeline_options.ocr_options = ocr_options
pipeline_options.generate_picture_images = True

doc_converter = DocumentConverter(
    format_options={
        InputFormat.IMAGE: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

def Run_Docling(Image_path):
    # Convert the image to Markdown
    start_time = time.time()

    result = doc_converter.convert(Image_path)

    Markdown = result.document.export_to_markdown()
    # Itirate over all pictures in the document
    for element, _level in result.document.iterate_items():
        if isinstance(element, PictureItem):
            filename = "Docling.jpg"
            #element.image.pil_image.show()
            element.image.pil_image.save(filename)
            extracted_text,Time = Run_Paddle(filename)
            Markdown = extracted_text+"\n\n"+ Markdown
    end_time = time.time()
    Inference_time = end_time - start_time
    return Markdown,Inference_time

import pandas as pd
from tqdm import tqdm
import os

def Run_Docling_ALL(Folder_path):
    df = pd.DataFrame(columns=['InvoiceName', 'Text', 'Time(s)'])
    Files = os.listdir(Folder_path)
    Texts = []
    Times = []
    for file in tqdm(Files,desc="Docling Processing", unit="number"):
        print(f"{file} is processing")
        Invoice_path = os.path.join(Folder_path, file)
        Text_docling,Time_docling = Run_Docling(Invoice_path)
        Texts.append(Text_docling)
        Times.append(Time_docling)
    df['InvoiceName'] = Files
    df['Text'] = Texts
    df['Time(s)'] = Times
    # Saving the DataFrame to a CSV file
    df.to_csv('CSVs/Docling_Results.csv', index=False)



[2025/01/18 17:32:09] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25,

#### Test

- Run on an Invoice

In [ ]:
Image_path = "Files/Test_Invoices/2.jpg"
Markdown,Inference_time = Run_Docling(Image_path)
print(Inference_time)
print(Markdown)

- Run on All Invoices

In [ ]:
Run_Docling_ALL("Files/Test_Invoices")

In [9]:
import pandas as pd
Docling_Results = pd.read_csv('CSVs/Docling_Results.csv')
Docling_Results.head()

,InvoiceName,Text,Time(s)
0,334.jpg,IMPRIMOS \n\n\nIMPRIMOS 24hPRINT .MA \n\n\n\n\...,11.526491
1,317.jpg,\n\n## Détail de la facture électricté\n\n| ...,11.056375
2,121.jpg,\n\n## BOXY DOM\n\nAdresse 10 Rue Chrarda Rdc ...,7.064245
3,66.jpg,IMIZEGH SARLU A PETE \n\n\n\n\n<!-- image -->\...,10.887913
4,2.jpg,KIPRINTSAEL IC0028b1004000053 \n\n\nKiPRINT BI...,9.063649


- Orginize the rows in accending order of the InvoiceName, this is important for the Evatuation logic.

In [12]:
import pandas as pd
import re

Docling_Results = pd.read_csv('CSVs/Docling_Results.csv')
Docling_Results['InvoiceNumber'] = Docling_Results['InvoiceName'].apply(lambda x: int(re.search(r'\d+', x).group()))
Docling_Results = Docling_Results.sort_values(by='InvoiceNumber')
Docling_Results = Docling_Results.reset_index(drop=True)
Docling_Results = Docling_Results.drop(columns=['InvoiceNumber'])

Docling_Results.to_csv('CSVs/Docling_Results.csv', index=False)


## LLM

- From a quick search, you can find that LLM via API Providers like OpenAI, Anthropic... are the best in Information extraction.
But since we decide to keep everything localy, we need to test some of the local LLMs with Oallama and Huggingface.

- For All the tested models i use the q-4 quantized version of the model.

#### Install Dependencies

In [13]:
exit()

In [ ]:
!ollama pull llama3.1:8b

In [12]:
!ollama list

NAME               ID              SIZE      MODIFIED           
llama3.1:8b        46e0c10c039e    4.9 GB    About a minute ago    
llava-llama3:8b    44c161b1f465    5.5 GB    42 minutes ago        
llama3.2:3b        a80c4f17acd5    2.0 GB    49 minutes ago        


#### LLM_Functions

In [13]:
################################################################################
################################ Structered Output ################################
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class Extraction_Facture(BaseModel):
    """Les Informations principales à extraire a partir d'une facture française"""
    Nom_facture: str = Field("", description="Le nom et le format(.jpg, .png, .jpeg) de l'image de la facture")
    Fournisseur: str = Field("", description="Le nom du fournisseur de la facture")
    date_facture: str = Field("", description="La date de la facture")
    num_facture: str = Field("", description="L'identifiant de la facture, peut être numérique ou alphanumérique")
    ttc: float = Field(0.0, description="Toutes Taxes Comprises. Il s'agit du prix final total payé par le client. Il inclut le prix Total HT et la Total TVA.")
    tt: float = Field(0.0, description="Le total hors taxes de la facture")
    tva: float = Field(0.0, description="Le Total taxes")



system = """
Vous êtes un assistant puissant dans l'extraction d'informations a partir des factures francaises. \n
- Un OCR est appliqué sur l'image de la facture, lire le text et extraire les informations suivantes :  \n

- Fournisseur, Date_Facture, Num_Facture, TTC, TT, TVA \n
- Les 3 prix TT,TTC,TVA doivent être numérique sans indication de devise. \n
- le nom de Fournisseur doit être en majuscule. \n
"""

## Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Le text donné par OCR ici :\n {OCR_Output} \n le Nom de facture est : \n {Nom_facture} \n"),
    ]
)

################################################################################
################################ LLM ###########################################
from langchain_openai import ChatOpenAI
import pandas as pd
import time

LLM = ChatOpenAI(
    model="llama3.1:8b",
    #temperature=0,
    api_key='ollama',
    base_url="http://localhost:11434/v1",
)
LLM_Structered = LLM.with_structured_output(Extraction_Facture)
LLM_Extractor = prompt | LLM_Structered

def Run_LLM(OCR_Output, Nom_facture):
    Input = {"OCR_Output":OCR_Output, "Nom_facture":Nom_facture}
    response = LLM_Extractor.invoke(Input)
    return response

# The Chain can raise an exception if the LLM fails to respect the structured output, so we can fix a retry number
RETRIES = 3

def Run_LLM_ALL(OCR_dataframe,pipelineName):
    LLM_Results = pd.DataFrame(columns=['Temps(s)','Nom_facture', 'Fournisseur', 'Date_Facture', 'Num_Facture', 'TTC', 'TT', 'TVA'])
    for index, row in OCR_dataframe.iterrows():
        print(f"Processing {index+1}/{len(OCR_dataframe)}...")
        for i in range(RETRIES):
            try:
                strat_time = time.time()
                response = Run_LLM(row['Text'], row['InvoiceName'])
                end_time = time.time()
                response_time = end_time - strat_time
                LLM_Results.loc[index] = [response_time,row['InvoiceName'], response.Fournisseur, response.date_facture, response.num_facture, response.ttc, response.tt, response.tva]
                #print(LLM_Results)
                break
            except Exception as e:
                #print(f"Error: {e}")
                LLM_Results.loc[index] = [0.0,row['InvoiceName'], '', '', '', 0.0, 0.0, 0.0]
                print(f"Retrying {i+1}/{RETRIES}...\n")

    # Saving the DataFrame to a CSV file
    LLM_Results.to_csv(f'CSVs/{pipelineName}_Results.csv', index=False)







#### Load OCR Results

In [14]:
import pandas as pd
df_docling = pd.read_csv('CSVs/Docling_Results.csv')
df_docling.head()

,InvoiceName,Text,Time(s)
0,2.jpg,KIPRINTSAEL IC0028b1004000053 \n\n\nKiPRINT BI...,9.063649
1,3.jpg,"FUNSBLACE 295,AnleBAcimcn et RuPersée,Etage NA...",9.046808
2,49.jpg,\n\n\n\n\n\naaaialati \n\n\nSgL \n\n\nslinna s...,17.714736
3,57.jpg,No Direct 0802002800 1.15Dh TtC/rmin depuis un...,10.475093
4,58.jpg,NETTO PAP PRODUITS DE NETTOYAGE ETD'HYGIENE \n...,9.243557


In [15]:
index = 0
Text_docling = df_docling['Text'].values[index]
InvoiceName = df_docling['InvoiceName'].values[index]

#print("Text_docling :", Text_docling)
print("InvoiceName :", InvoiceName)

response = Run_LLM(Text_docling, InvoiceName)
response

InvoiceName : 2.jpg


Extraction_Facture(Nom_facture='2.jpg', Fournisseur='OUTIDIS SARL', date_facture='13/01/2023', num_facture='FV230007', ttc=586.8, tt=489.0, tva=97.8)

#### Run LLM on All Invoices

In [16]:
pipelineName = "llama3.1_8b_Docling"
Run_LLM_ALL(df_docling,pipelineName)
#pipelineName = "Qwen2.5_7b_Easy"
#Run_LLM_ALL(df_easyocr,pipelineName)

Processing 1/30...
Processing 2/30...
Processing 3/30...
Retrying 1/3...

Processing 4/30...
Processing 5/30...
Processing 6/30...
Processing 7/30...
Processing 8/30...
Processing 9/30...
Processing 10/30...
Processing 11/30...
Processing 12/30...
Processing 13/30...
Processing 14/30...
Retrying 1/3...

Processing 15/30...
Processing 16/30...
Processing 17/30...
Processing 18/30...
Processing 19/30...
Retrying 1/3...

Processing 20/30...
Retrying 1/3...

Processing 21/30...
Processing 22/30...
Processing 23/30...
Retrying 1/3...

Retrying 2/3...

Processing 24/30...
Processing 25/30...
Processing 26/30...
Processing 27/30...
Retrying 1/3...

Processing 28/30...
Processing 29/30...
Processing 30/30...


#### Evaluation Function

In [17]:
import Levenshtein
import pandas as pd

def Calculate_Similarity(string1, string2):
    if pd.isna(string1):
        return 0
    distance = Levenshtein.distance(string1, string2)
    score = 1-(distance/max(len(string1), len(string2)))
    return score

def Evaluate(LLM_Results, Ground_Truth):
    Total_Similarity = 0
    Total_Margin = 0
    Total_Time = 0
    for index, Ground_row in Ground_Truth.iterrows():
        print("######## Invoice Number : ", index)
        Line_Similarity = 0
        Line_margin = 0

        # Strings Similarities
        Line_Similarity = Calculate_Similarity(LLM_Results.loc[index,'Fournisseur'], Ground_row['Fournisseur']) + Calculate_Similarity(LLM_Results.loc[index,'Date_Facture'], Ground_row['Date_Facture']) + Calculate_Similarity(LLM_Results.loc[index,'Num_Facture'], Ground_row['Num_Facture'])
        Total_Similarity += Line_Similarity/3
        print("Total Similarity : ", Total_Similarity)

        # Prices Margins
        Line_margin = abs(LLM_Results.loc[index,'TTC'] - Ground_row['TTC']) + abs(LLM_Results.loc[index,'TT'] - Ground_row['TT']) + abs(LLM_Results.loc[index,'TVA'] - Ground_row['TVA'])
        Total_Margin += Line_margin
        print("Total Margin : ", Total_Margin)

        Total_Time += LLM_Results.loc[index,'Temps(s)']

    Accuarcy = Total_Similarity/len(Ground_Truth)
    Average_Time = Total_Time/len(Ground_Truth)
    Average_Margin = Total_Margin/len(Ground_Truth)
    return Accuarcy, Average_Time, Average_Margin


#### Evaluation of Performance

In [18]:
LLM_Results = pd.read_csv(f'CSVs/{pipelineName}_Results.csv')
Ground_Truth = pd.read_csv('Files/grounding_truth.csv')
Accuarcy, Average_Time, Average_Margin = Evaluate(LLM_Results, Ground_Truth)

######## Invoice Number :  0
Total Similarity :  1.0
Total Margin :  0.0
######## Invoice Number :  1
Total Similarity :  1.288888888888889
Total Margin :  0.0
######## Invoice Number :  2
Total Similarity :  2.2888888888888888
Total Margin :  0.0
######## Invoice Number :  3
Total Similarity :  3.2888888888888888
Total Margin :  0.0
######## Invoice Number :  4
Total Similarity :  4.016161616161616
Total Margin :  0.0
######## Invoice Number :  5
Total Similarity :  4.692274976485503
Total Margin :  0.0
######## Invoice Number :  6
Total Similarity :  5.075608309818836
Total Margin :  0.0
######## Invoice Number :  7
Total Similarity :  5.867274976485503
Total Margin :  0.0
######## Invoice Number :  8
Total Similarity :  6.585223694434221
Total Margin :  0.0
######## Invoice Number :  9
Total Similarity :  7.378874488085015
Total Margin :  0.0
######## Invoice Number :  10
Total Similarity :  8.272036881247407
Total Margin :  1000.0
######## Invoice Number :  11
Total Similarity :  8

In [19]:
print("Accuarcy :(%) ", Accuarcy*100)
print("Average_Time(s) : ", Average_Time)
print("Average_Margin (DH) : ", Average_Margin)

Accuarcy :(%)  65.18576994664701
Average_Time(s) :  4.3746187845865885
Average_Margin (DH) :  18175.099113333334


In [20]:
Results = pd.read_csv('CSVs/Results.csv')
Results

,PipelineName,Accuarcy (%),Average_Time (s),Average_Margin (DH)
0,llama3.2_3b_Paddle,69.997657,1.532643,7621.832167


In [21]:
Results.loc[len(Results)] = [pipelineName,Accuarcy*100, Average_Time, Average_Margin]
Results.to_csv('CSVs/Results.csv', index=False)

In [22]:
Results

,PipelineName,Accuarcy (%),Average_Time (s),Average_Margin (DH)
0,llama3.2_3b_Paddle,69.997657,1.532643,7621.832167
1,llama3.1_8b_Docling,65.185770,4.374619,18175.099113


# Results Refining

- Because we forget to add the average time of the Engine used (Paddle, Easy, Docling, etc.), we will now adjust the results accordingly.

In [23]:
import pandas as pd

df_paddle = pd.read_csv("CSVs/PaddleOCR_Results.csv")
df_easy = pd.read_csv("CSVs/EasyOCR_Results.csv")
df_docling = pd.read_csv("CSVs/Docling_Results.csv")

In [24]:
paddle_time_mean = df_paddle['Time(s)'].mean()
easy_time_mean = df_easy['Time(s)'].mean()
docling_time_mean = df_docling['Time(s)'].mean()

print(f"Paddle Time Mean: {paddle_time_mean}")
print(f"Easy Time Mean: {easy_time_mean}")
print(f"Docling Time Mean: {docling_time_mean}")

Paddle Time Mean: 2.906348546346029
Easy Time Mean: 2.905944999059041
Docling Time Mean: 11.321790130933126


In [26]:
Results = pd.read_csv("CSVs/Results.csv")
Results

,PipelineName,Accuarcy (%),Average_Time (s),Average_Margin (DH)
0,llama3.2_3b_Paddle,69.997657,1.532643,7621.832167
1,llama3.1_8b_Docling,65.185770,4.374619,18175.099113


In [27]:
Engine_names = []
Efficiency = []
Model_names = []

for index, row in Results.iterrows():  # `index` is the row index in the DataFrame
    engine_name = row["PipelineName"].split("_")[-1]
    model_name = "_".join(row["PipelineName"].split("_")[:-1])

    Engine_names.append(engine_name)
    Model_names.append(model_name)

    # Modify the DataFrame directly using the index
    if engine_name == "Paddle":
        Results.at[index, "Average_Time (s)"] += paddle_time_mean
    elif engine_name == "Easy":
        Results.at[index, "Average_Time (s)"] += easy_time_mean
    else:
        Results.at[index, "Average_Time (s)"] += docling_time_mean

    efficiency = Results.at[index, "Accuarcy (%)"]/Results.at[index, "Average_Time (s)"]
    Efficiency.append(efficiency)


In [28]:
Results["Engine"] = Engine_names
Results["Model"] = Model_names
Results["Efficiency"] = Efficiency
Results

,PipelineName,Accuarcy (%),Average_Time (s),Average_Margin (DH),Engine,Model,Efficiency
0,llama3.2_3b_Paddle,69.997657,4.438992,7621.832167,Paddle,llama3.2_3b,15.768819
1,llama3.1_8b_Docling,65.185770,15.696409,18175.099113,Docling,llama3.1_8b,4.152910
